# EKS Ops Agent Workshop

Build an AI agent that manages and troubleshoots Amazon EKS clusters using LangGraph, MCP Server and kagent.

| Module | Description | Dependency |
|--------|-------------|------------|
| **Module 0** | Provision an EKS cluster with kagent enabled | |
| **Module 1** | Barebone agent - Build and deploy BYO agent with Amazon Bedrock as model provider using kagent | Module 0 |
| **Module 2** | EKS MCP Server integration - Connect the agent to EKS MCP Server and access tools for cluster operations | Module 1 |
| **Module 3** | Memory - Persistent memory with Redis for user defaults | Module 1 or Module 2 |

## Architecture

```
┌──────────────────────────────────────────────────────────────────────────────┐
│                              EKS Cluster                                     │
│                                                                              │
│  ┌────────────────────────────────────────────────────────────────────────┐  │
│  │                         kagent namespace                               │  │
│  │                                                                        │  │
│  │   ┌─────────────┐     ┌───────────────────┐     ┌───────────────────┐  │  │
│  │   │  kagent-ui  │◄───►│ kagent-controller │◄───►│   eks-ops-agent   │  │  │
│  │   │    (Pod)    │     │       (Pod)       │     │       (Pod)       │  │  │
│  │   └─────────────┘     └─────────┬─────────┘     └─────────┬─────────┘  │  │
│  │                                 │                         │            │  │
│  │                       ┌─────────▼─────────┐               │            │  │
│  │                       │   PostgreSQL /    │               │            │  │
│  │                       │   SQLite (DB)     │               │            │  │
│  │                       └───────────────────┘               │            │  │
│  │                                                           │            │  │
│  │                       ┌───────────────────┐               │            │  │
│  │                       │  Redis (Module 3) │◄──────────────┘            │  │
│  │                       │  (User Defaults)  │                            │  │
│  │                       └───────────────────┘                            │  │
│  └────────────────────────────────────────────────────────────────────────┘  │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘
                                      │
          ┌───────────────────────────┼───────────────────────────┐
          │                           │                           │
          ▼                           ▼                           ▼
 ┌─────────────────┐        ┌─────────────────┐        ┌─────────────────┐
 │  Amazon Bedrock │        │   EKS MCP       │        │  Kubernetes     │
 │    (Claude)     │        │   Server        │        │     API         │
 └─────────────────┘        └─────────────────┘        └─────────────────┘
```

| Component | Role |
|-----------|------|
| **kagent-ui** | Web interface for chatting with agents |
| **kagent-controller** | Manages agent lifecycle, routes messages, stores sessions |
| **eks-ops-agent** | Your LangGraph agent (BYO agent pattern) |
| **PostgreSQL/SQLite** | Stores chat sessions, agent state (short-term memory) |
| **Redis** | Stores user defaults across sessions (long-term memory - Module 3) |
| **Amazon Bedrock** | LLM provider (Claude) for agent reasoning |
| **EKS MCP Server** | AWS-managed API providing 20+ EKS tools |

---
## Prerequisites

- **EC2 cloud desktop** is already set up and you can log in using the DCV Viewer. The desktop has Docker, AWS CLI, kubectl, terraform, VSCode and Kiro pre-installed.
- **No GPU instances required.** Standard CPU instances are sufficient for all modules.
- **AWS account with Bedrock access** — Enable model access in [AWS Bedrock Console](https://console.aws.amazon.com/bedrock/) → Model access. Default model: Claude Sonnet 4.

---
## Module 0: Provision an EKS Cluster

In this module, you will provision an EKS cluster with kagent enabled. At the end of the module, you will have:
- VPC with public/private subnets across 3 AZs
- EKS cluster with system node group
- kagent controller and UI deployed
- IAM role for Bedrock access

### Step 0.1: Clone Repository

In [ ]:
%%bash
cd ~
if [ ! -d "amazon-eks-machine-learning-with-terraform-and-kubeflow" ]; then
    git clone https://github.com/aws-samples/amazon-eks-machine-learning-with-terraform-and-kubeflow.git
    echo "Repository cloned successfully."
else
    echo "Repository already exists. Pulling latest changes."
    cd amazon-eks-machine-learning-with-terraform-and-kubeflow && git pull
fi

### Step 0.2: Configure Variables

Update the values below for your environment. The `S3_BUCKET` must be globally unique and the `CLUSTER_NAME` must be less than 16 characters.

In [ ]:
import os

# =================================================================
# UPDATE THESE VALUES
# =================================================================
S3_BUCKET = "<your-bucket-name>"  # Must be globally unique
S3_PREFIX = "eks-agent"
AWS_REGION = "us-west-2"
CLUSTER_NAME = "my-cluster"  # Must be less than 16 characters
AZS = '["us-west-2a","us-west-2b","us-west-2c"]'
# =================================================================

os.environ["S3_BUCKET"] = S3_BUCKET
os.environ["S3_PREFIX"] = S3_PREFIX
os.environ["AWS_REGION"] = AWS_REGION
os.environ["CLUSTER_NAME"] = CLUSTER_NAME
os.environ["AZS"] = AZS

REPO_DIR = os.path.expanduser("~/amazon-eks-machine-learning-with-terraform-and-kubeflow")
os.environ["REPO_DIR"] = REPO_DIR
AGENT_DIR = os.path.join(REPO_DIR, "examples/agentic/eks-ops-agent")
os.environ["AGENT_DIR"] = AGENT_DIR
TF_DIR = os.path.join(REPO_DIR, "eks-cluster/terraform/aws-eks-cluster-and-nodegroup")
os.environ["TF_DIR"] = TF_DIR

# Validate
if S3_BUCKET == "<your-bucket-name>":
    print("Error: Please update S3_BUCKET with your own bucket name!")
elif len(CLUSTER_NAME) > 16:
    print("Error: Cluster name cannot exceed 16 characters!")
else:
    print(f"S3 Bucket:    {S3_BUCKET}")
    print(f"Region:       {AWS_REGION}")
    print(f"Cluster Name: {CLUSTER_NAME}")
    print(f"Repo Dir:     {REPO_DIR}")
    print(f"Agent Dir:    {AGENT_DIR}")
    print(f"TF Dir:       {TF_DIR}")

### Step 0.3: Create S3 Bucket

In [ ]:
!aws s3 mb s3://$S3_BUCKET --region $AWS_REGION

### Step 0.4: Configure Terraform Backend

Configure S3 backend for Terraform state storage.

In [ ]:
%%bash
cd $REPO_DIR
./eks-cluster/utils/s3-backend.sh $S3_BUCKET $S3_PREFIX

### Step 0.5: Create Terraform Variables

Create a `workshop.tfvars` file to enable kagent. All other optional components (Karpenter, Kubeflow, etc.) are disabled by default.

In [ ]:
%%bash
cat <<'EOF' > $TF_DIR/workshop.tfvars
# Enable kagent
kagent_enabled               = true
kagent_database_type         = "sqlite"
kagent_enable_ui             = true
kagent_enable_bedrock_access = true
EOF
echo "Created workshop.tfvars"

### Step 0.6: Run Setup Script

The setup script adds IAM permissions to your EC2 instance role that Terraform needs to create the kagent Bedrock access role.

In [ ]:
%cd $AGENT_DIR
!./setup.sh

### Step 0.7: Install kubectl

In [ ]:
%%bash
$REPO_DIR/eks-cluster/utils/install-kubectl-linux.sh

In [ ]:
!kubectl version --client

### Step 0.8: Initialize and Apply Terraform

This step takes **15-20 minutes**. It creates the EKS cluster, VPC, storage, and deploys kagent.

In [ ]:
%%bash
docker logout public.ecr.aws 2>/dev/null
cd $TF_DIR
terraform init -reconfigure

In [ ]:
%%bash
cd $TF_DIR
START=$(date +%s)
terraform apply -auto-approve \
  -var="profile=default" \
  -var="region=$AWS_REGION" \
  -var="cluster_name=$CLUSTER_NAME" \
  -var="azs=$AZS" \
  -var="import_path=s3://$S3_BUCKET/$S3_PREFIX/" \
  -var-file=workshop.tfvars \
  > tf_apply.log 2>&1
RC=$?
ELAPSED=$(( $(date +%s) - START ))

echo "=== Last 30 lines ==="
tail -30 tf_apply.log
echo ""
echo "terraform apply completed in ${ELAPSED}s"

if [ $RC -ne 0 ]; then
  echo "FAILED - check full log: cat $TF_DIR/tf_apply.log"
  exit 1
else
  echo "SUCCESS"
fi

### Step 0.9: Verify Cluster and kagent

Update kubeconfig and check that kagent pods are running. You should see 15+ pods including `kagent-controller` and `kagent-ui`.

> **Note:** kagent comes with several built-in agents (k8s-agent, helm-agent, istio-agent, etc.). In this workshop, you'll build and deploy your own **eks-ops-agent** alongside them.

In [ ]:
!aws eks update-kubeconfig --name $CLUSTER_NAME --region $AWS_REGION
!kubectl get pods -n kagent

---
## Module 1: Barebone Agent

In this module, you'll build and deploy a simple Q&A agent that can answer Kubernetes and EKS questions using Amazon Bedrock Claude. The agent's default model is `us.anthropic.claude-sonnet-4-20250514-v1:0` (Claude Sonnet 4).

### Step 1.1: Review the LangGraph Agent Code

The agent uses a ReAct pattern (agent -> tools -> agent loop) with `ChatBedrockConverse` for LLM calls.

In [ ]:
!cat $AGENT_DIR/src/agent.py

### Step 1.2: Review the kagent Wrapper

The `KAgentApp` wrapper handles the A2A protocol between kagent controller and the LangGraph agent.

In [ ]:
!cat $AGENT_DIR/src/app.py

### Step 1.3: Build and Deploy the Agent

The script will:
1. Build the container image (`docker build`)
2. Create ECR repository and push the image
3. Apply the Agent CRD manifest to kagent
4. Annotate ServiceAccount for IRSA (IAM Roles for Service Accounts)
5. Wait until agent pods are ready

In [ ]:
%cd $AGENT_DIR
!./build-and-deploy.sh

### Step 1.4: Access the kagent UI

Expose the kagent UI via a LoadBalancer. Update `MY_IP` with your IP address from https://checkip.amazonaws.com/

In [ ]:
%%bash
MY_IP="<your-ip-address>"

kubectl patch svc kagent-ui -n kagent -p "{
  \"spec\":{
    \"type\":\"LoadBalancer\",
    \"loadBalancerSourceRanges\":[\"${MY_IP}/32\"]
  }
}"

echo "Waiting for LoadBalancer..."
sleep 15

LB_URL=$(kubectl get svc kagent-ui -n kagent -o jsonpath='{.status.loadBalancer.ingress[0].hostname}')
echo ""
echo "kagent UI: http://${LB_URL}:8080"
echo "Restricted to: ${MY_IP}/32"

> **Note:** To revert back to ClusterIP after testing:
> ```
> !kubectl patch svc kagent-ui -n kagent -p '{"spec":{"type":"ClusterIP"}}'
> ```

### Step 1.5: Test the Agent

Open the LoadBalancer URL (port 8080), select **eks-ops-agent**, and try these prompts:

- "What is a Kubernetes Pod?"
- "How do I troubleshoot a CrashLoopBackOff error?"
- "Explain the difference between a Deployment and a StatefulSet"

The agent should respond with helpful Kubernetes/EKS guidance (but cannot access your actual cluster yet - that comes in Module 2).

### Verify: Check Agent Logs

In [ ]:
!kubectl logs -n kagent -l kagent=eks-ops-agent --tail=20

---
## Module 2: EKS MCP Server Integration

In this module, you'll enable 20+ tools from the [AWS managed EKS MCP Server](https://docs.aws.amazon.com/eks/latest/userguide/eks-mcp.html) that allow the agent to query and manage your actual EKS cluster.

**How this works:** The MCP integration code is already in `src/tools.py`. Setting `ENABLE_MCP_TOOLS=true` tells the agent to load these tools at startup.

### Step 2.1: Understand the Code

The MCP integration is in `src/tools.py`. Key points:

- Uses `langchain_mcp_adapters.client.MultiServerMCPClient` to connect to EKS MCP Server
- Spawns `mcp-proxy-for-aws` as a subprocess via stdio transport
- IRSA credentials are passed through automatically
- The agent uses `ChatBedrockConverse` (Converse API) for tool result handling

```python
# src/tools.py - Connection to EKS MCP Server
def get_mcp_server_config() -> dict:
    eks_mcp_endpoint = f"https://eks-mcp.{config.AWS_REGION}.api.aws/mcp"
    return {
        "eks-mcp": {
            "transport": "stdio",
            "command": "uvx",
            "args": ["mcp-proxy-for-aws@latest", eks_mcp_endpoint,
                     "--service", "eks-mcp", "--region", config.AWS_REGION],
            "env": {"AWS_REGION": config.AWS_REGION,
                    **{k: v for k, v in os.environ.items() if k.startswith("AWS_")}},
        }
    }
```

### Step 2.2: Enable MCP Tools

Update the manifest to set `ENABLE_MCP_TOOLS` to `"true"`.

In [ ]:
%cd $AGENT_DIR
!sed -i '/ENABLE_MCP_TOOLS/{n;s/value: "false"/value: "true"/}' manifests/eks-ops-agent.yaml

# Verify the change
!grep -A1 ENABLE_MCP_TOOLS manifests/eks-ops-agent.yaml

### Step 2.3: Redeploy the Agent

In [ ]:
%cd $AGENT_DIR
!./build-and-deploy.sh

### Step 2.4: Verify Tools Loaded

You should see:
```
INFO - Memory disabled (set ENABLE_MEMORY=true to enable)
INFO - Creating agent with 20 tools
INFO - Starting EKS Ops Agent on 0.0.0.0:8080
```

The key confirmation is **"Creating agent with 20 tools"**.

In [ ]:
!kubectl logs -n kagent -l kagent=eks-ops-agent --tail=20

### Step 2.5: Test MCP Tools

Open the kagent UI and try these prompts (replace `<cluster-name>` with your actual cluster name):

1. **List resources:**
   ```
   List all pods in the kagent namespace on cluster <cluster-name>
   ```

2. **Get pod logs:**
   ```
   Get the logs from pod <pod-name> in namespace <namespace> on cluster <cluster-name>
   ```

3. **Check cluster events:**
   ```
   Show me recent events in the kube-system namespace on cluster <cluster-name>
   ```

4. **Generate and deploy manifests:**
   ```
   Generate a deployment manifest for a Redis application with 2 replicas.
   Deploy the manifest, ensure that Pods are in Running state.
   ```

5. **Get cluster insights:**
   ```
   Get insights and recommendations for cluster <cluster-name>
   ```

6. **Multi-step deployment task:**
   ```
   On cluster <cluster-name>, deploy a new nginx application called "test-nginx" with 3 replicas
   in the default namespace. After deployment, verify all pods are running.
   Then scale it down to 1 replica and confirm the change.
   ```

7. **Cluster discovery** (agent discovers clusters automatically):
   ```
   List all pods in the default namespace
   ```

### Sample Troubleshooting Scenario

This demonstrates the agent's ability to diagnose issues in real-time.

**Step 1:** In the kagent UI, ask:
```
Deploy an nginx app called "broken-app" using image "nginx:doesnotexist" with 2 replicas
to the default namespace on cluster <cluster-name>
```

**Step 2:** Then ask:
```
The broken-app pods are not running. Investigate why and tell me how to fix it.
```

The agent will use multiple tools to diagnose:
- `list_k8s_resources` - check pod status
- `get_k8s_events` - find error events
- `search_eks_troubleshooting_guide` - look up the error
- Provide actionable fix recommendations

---
## Module 3: Memory with Redis

In this module, you'll enable memory that persists user defaults (cluster, namespace) across chat sessions.

**How this works:** The memory code is already in `src/memory.py`. Setting `ENABLE_MEMORY=true` and deploying Redis enables this feature.

```
Session 1:
  User: "Set my default cluster to <cluster-name> and namespace to default"
  Agent: Saved defaults

Session 2 (new session):
  User: "List all pods"
  Agent: (retrieves defaults from Redis, uses <cluster-name>/default)
```

> **Note:** This module uses an in-cluster Redis deployment for simplicity. For production, consider Amazon ElastiCache, MemoryDB, OpenSearch, or DynamoDB.

### Step 3.1: Understand the Code

The memory implementation is in `src/memory.py`. Key pattern:

```python
class MemoryService:
    async def get_defaults(self, user_id: str) -> UserDefaults:
        client = await self._get_client()
        data = await client.get(f"user:{user_id}:defaults")
        return UserDefaults.from_dict(json.loads(data)) if data else UserDefaults()

    async def set_defaults(self, user_id: str, cluster: str, namespace: str) -> UserDefaults:
        client = await self._get_client()
        defaults = UserDefaults(cluster=cluster, namespace=namespace)
        await client.set(f"user:{user_id}:defaults", json.dumps(defaults.to_dict()))
        return defaults
```

Memory tools (`set_user_defaults`, `get_user_defaults`, `clear_user_defaults`) are exposed to the agent so it can get/set defaults on behalf of the user.

### Step 3.2: Deploy Redis

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: redis
  namespace: kagent
spec:
  replicas: 1
  selector:
    matchLabels:
      app: redis
  template:
    metadata:
      labels:
        app: redis
    spec:
      containers:
      - name: redis
        image: redis:7-alpine
        ports:
        - containerPort: 6379
---
apiVersion: v1
kind: Service
metadata:
  name: redis
  namespace: kagent
spec:
  selector:
    app: redis
  ports:
  - port: 6379
    targetPort: 6379
EOF

In [ ]:
# Verify Redis is running
!kubectl get pods -n kagent -l app=redis

### Step 3.3: Enable Memory

Update the manifest to set `ENABLE_MEMORY` to `"true"`.

> **Note:** `REDIS_URL` is already set to `redis://redis.kagent.svc.cluster.local:6379` in the manifest.

In [ ]:
%cd $AGENT_DIR
!sed -i '/ENABLE_MEMORY/{n;s/value: "false"/value: "true"/}' manifests/eks-ops-agent.yaml

# Verify the change
!grep -A1 ENABLE_MEMORY manifests/eks-ops-agent.yaml

### Step 3.4: Rebuild and Deploy

In [ ]:
%cd $AGENT_DIR
!./build-and-deploy.sh

### Step 3.5: Verify Memory Loaded

You should see:
```
INFO - Loaded 20 EKS MCP tools
INFO - Memory enabled (Redis: redis://redis.kagent.svc.cluster.local:6379)
INFO - Loaded 3 memory tools
INFO - Creating agent with 23 tools
```

The agent now has 23 tools (20 MCP + 3 memory).

In [ ]:
!kubectl logs -n kagent -l kagent=eks-ops-agent --tail=20

### Step 3.6: Test Memory

Open the kagent UI and try these prompts (replace `<cluster-name>` with your actual cluster name):

1. **Set defaults:**
   ```
   Set my default cluster to <cluster-name> and namespace to default
   ```

2. **Verify defaults saved:**
   ```
   What are my defaults?
   ```

3. **Start a new chat session** (click "New Chat" in kagent UI)

4. **Use defaults implicitly:**
   ```
   List all pods
   ```
   The agent should use your saved cluster and namespace without asking.

5. **Clear defaults:**
   ```
   Clear my defaults
   ```

---
## Troubleshooting

| Error | Cause | Solution |
|-------|-------|----------|
| `AccessDenied on InvokeModel` | Bedrock model not enabled | Enable model access in [Bedrock console](https://console.aws.amazon.com/bedrock/) |
| `AccessDenied on InvokeModel` (Claude 4.x) | Missing inference profile prefix | Use `us.` prefix (e.g., `us.anthropic.claude-sonnet-4-...`) |
| `MCP tools not loading` | Environment variable not set | Verify `ENABLE_MCP_TOOLS=true` in manifest |
| `MCP tools not loading` | Missing IAM permissions | Check IAM role has `eks-mcp:*` permissions |
| `Tool calls failing with Unauthorized` | Missing EKS Access Entry | Run `aws eks list-access-entries --cluster-name <cluster>` to verify |
| `IRSA not working` | ServiceAccount not annotated | Check: `kubectl get sa eks-ops-agent -n kagent -o yaml` |

In [ ]:
# Useful troubleshooting commands
!kubectl get agents -n kagent
!kubectl get pods -n kagent -l kagent=eks-ops-agent
!kubectl logs -n kagent -l kagent=eks-ops-agent --tail=50